In [1]:
import carla
import random
import math
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.behavior_agent import BehaviorAgent

In [2]:
# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()
client.load_world('Town01')

In [3]:
ego_bp = world.get_blueprint_library().find('vehicle.dodge.charger_2020')
ego_bp.set_attribute('role_name','ego')

In [5]:
def a_star_search(start_wp, end_wp, heuristic):
    open_set = set([start_wp])
    came_from = {}
    g_score = {start_wp: 0}
    f_score = {start_wp: heuristic(start_wp, end_wp)}
    while open_set:
        current = min(open_set, key=lambda wp: f_score[wp])
        if current == end_wp:
            return reconstruct_path(came_from, current)
        open_set.remove(current)
        for neighbor in current.next(2.0):  # 2 meters forward
            tentative_g_score = g_score[current] + distance(current, neighbor)
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, end_wp)
                open_set.add(neighbor)
    return []

def heuristic(wp1, wp2):
    return wp1.transform.location.distance(wp2.transform.location)

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    total_path.reverse()
    return total_path

def distance(wp1, wp2):
    loc1 = wp1.transform.location
    loc2 = wp2.transform.location
    return math.sqrt((loc1.x - loc2.x)**2 + (loc1.y - loc2.y)**2 + (loc1.z - loc2.z)**2)

def generate_random_route():
    map = world.get_map()
    waypoints = map.generate_waypoints(distance=20.0)  # Generate waypoints every 20 meters
    start_wp = random.choice(waypoints)
    end_wp = random.choice(waypoints)
    route = a_star_search(start_wp, end_wp, heuristic)
    return route

def follow_vehicle(client, vehicle, offset=carla.Location(x=-6, z=2), pitch=-15):
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    location = transform.location + transform.get_forward_vector() * offset.x + carla.Location(z=offset.z)
    rotation = carla.Rotation(pitch=pitch, yaw=transform.rotation.yaw + 0, roll=0)
    spectator.set_transform(carla.Transform(location, rotation))

In [6]:
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [7]:
amap = world.get_map()
sampling_resolution = 2
grp = GlobalRoutePlanner(amap, sampling_resolution)
spawn_points = world.get_map().get_spawn_points()
a = carla.Location(spawn_points[50].location)
b = carla.Location(spawn_points[100].location)
w1 = grp.trace_route(a, b) # there are other funcations can be used to generate a route in GlobalRoutePlanner.
i = 0
for w in w1:
    if i % 10 == 0:
        world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=120.0,
        persistent_lines=True)
    else:
        world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
        color = carla.Color(r=0, g=0, b=255), life_time=1000.0,
        persistent_lines=True)
    i += 1

In [8]:
ego_transform = spawn_points[50]
ego_vehicle = world.spawn_actor(ego_bp,ego_transform)
print('\nEgo is spawned')


Ego is spawned


In [9]:
follow_vehicle(client, ego_vehicle)

In [11]:
# To start a basic agent
agent = BasicAgent(ego_vehicle)
destination = spawn_points[100].location
agent.set_destination(destination)

In [ ]:
while True:
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
    ego_vehicle.apply_control(agent.run_step())
    follow_vehicle(client, ego_vehicle)